In [1]:
import yfinance as yf
import pandas as pd 
import numpy as np
import pandas_ta as ta

dataF = yf.download("EURUSD=X", start="2023-10-13", end="2023-12-01", interval='5m')
dataF.iloc[:,:]

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2023-10-13 00:00:00+01:00,1.053408,1.053519,1.053408,1.053408,1.053408,0
2023-10-13 00:05:00+01:00,1.053519,1.053519,1.053519,1.053519,1.053519,0
2023-10-13 00:10:00+01:00,1.053630,1.053630,1.053519,1.053519,1.053519,0
2023-10-13 00:15:00+01:00,1.053519,1.053630,1.053519,1.053630,1.053630,0
2023-10-13 00:20:00+01:00,1.053630,1.053630,1.053630,1.053630,1.053630,0
...,...,...,...,...,...,...
2023-11-30 23:35:00+00:00,1.089443,1.089681,1.089443,1.089681,1.089681,0
2023-11-30 23:40:00+00:00,1.089681,1.089681,1.089681,1.089681,1.089681,0
2023-11-30 23:45:00+00:00,1.089681,1.089681,1.089681,1.089681,1.089681,0


In [2]:
grid_distance = 0.005
TPSL_Ratio = 1.5
midprice = 1.065

In [3]:
def generate_grid(midprice, grid_distance, grid_range):
    return (np.arange(midprice-grid_range, midprice+grid_range, grid_distance))

grid = generate_grid(midprice=midprice, grid_distance=grid_distance, grid_range=0.1)
grid

array([0.965, 0.97 , 0.975, 0.98 , 0.985, 0.99 , 0.995, 1.   , 1.005,
       1.01 , 1.015, 1.02 , 1.025, 1.03 , 1.035, 1.04 , 1.045, 1.05 ,
       1.055, 1.06 , 1.065, 1.07 , 1.075, 1.08 , 1.085, 1.09 , 1.095,
       1.1  , 1.105, 1.11 , 1.115, 1.12 , 1.125, 1.13 , 1.135, 1.14 ,
       1.145, 1.15 , 1.155, 1.16 , 1.165])

In [4]:
signal = [0]*len(dataF)
i = 0 
for index, row in dataF.iterrows():
    for p in grid:
        if min(row.Low, row.High) < p and max(row.Low, row.High) > p:
            signal[i] = 1
    i+=1

In [5]:
dataF["signal"] = signal 
dataF[dataF["signal"] == 1]

,Open,High,Low,Close,Adj Close,Volume,signal
Datetime,,,,,,,
2023-10-13 04:00:00+01:00,1.054964,1.055075,1.054964,1.055075,1.055075,0,1
2023-10-13 04:05:00+01:00,1.055075,1.055075,1.054964,1.055075,1.055075,0,1
2023-10-13 05:40:00+01:00,1.054964,1.055075,1.054964,1.055075,1.055075,0,1
2023-10-13 06:35:00+01:00,1.055298,1.055298,1.054964,1.054964,1.054964,0,1
2023-10-13 06:55:00+01:00,1.054852,1.055075,1.054852,1.055075,1.055075,0,1
...,...,...,...,...,...,...,...
2023-11-30 17:40:00+00:00,1.089918,1.090156,1.089918,1.090156,1.090156,0,1
2023-11-30 17:45:00+00:00,1.090037,1.090037,1.089562,1.089562,1.089562,0,1
2023-11-30 18:10:00+00:00,1.089800,1.090037,1.089800,1.090037,1.090037,0,1


In [6]:
dfpl = dataF[:].copy()
def SIGNAL():
    return dfpl.signal
dfpl['ATR'] = ta.atr(high=dfpl.High, low=dfpl.Low, close=dfpl.Close, length=14)

In [9]:
from backtesting import Strategy, Backtest
import backtesting

class MyStrat(Strategy):
    mysize = 0.99
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)
    
    def next(self):
        super().next()
        slatr = self.data.ATR[-1]#grid_distance
        TPSLRatio = TPSL_Ratio

        if self.signal1==1 and len(self.trades) <= 2:
            sl1 = self.data.Close[-1] - slatr
            tp1 = self.data.Close[-1] + slatr*TPSLRatio
            self.buy(sl=sl1, tp=tp1, size=self.mysize)

            sl1 = self.data.Close[-1] + slatr
            tp1 = self.data.Close[-1] - slatr*TPSLRatio
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

bt = Backtest(dfpl, MyStrat, cash=1000, margin=1/2, commission=0.000)
stat = bt.run()
stat

Start                     2023-10-13 00:00...
End                       2023-11-30 23:55...
Duration                     49 days 00:55:00
Exposure Time [%]                    94.18628
Equity Final [$]                  1055.774656
Equity Peak [$]                   1082.714678
Return [%]                           5.577466
Buy & Hold Return [%]                3.420842
Return (Ann.) [%]                      55.463
Volatility (Ann.) [%]               22.203976
Sharpe Ratio                         2.497886
Sortino Ratio                         7.41398
Calmar Ratio                        18.873175
Max. Drawdown [%]                   -2.938721
Avg. Drawdown [%]                   -0.240277
Max. Drawdown Duration        8 days 04:55:00
Avg. Drawdown Duration        0 days 10:06:00
# Trades                                   78
Win Rate [%]                        42.307692
Best Trade [%]                        0.77637
Worst Trade [%]                     -0.536412
Avg. Trade [%]                    

In [10]:
backtesting.set_bokeh_output(notebook=False)
bt.plot(show_legend=False, plot_width=None, plot_equity=True, plot_return=False,
plot_pl = False, plot_volume=False, plot_drawdown = False, smooth_equity = False,
superimpose=True, resample=False, reverse_indicators=False, open_browser=True)


d:\Python\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
d:\Python\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
d:\Python\lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
d:\Python\lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p1252', ...)